In [1]:
import sys,os,datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
from __future__ import print_function

In [2]:
print(np.__version__);
print(pd.__version__);

1.12.1
0.19.2


In [3]:
import  cv2 as cv
cv.__version__

'3.2.0'

In [4]:
sys.path.append('../Python')
from helper import formFH, paths_input

In [5]:
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential,save_model,load_model
from keras.layers import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers.normalization import BatchNormalization
import keras.optimizers
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [6]:
keras.__version__

'2.0.4'

In [7]:
trLabels,trDirTIF,trDirJPG,teDirTIF,teDirJPG = paths_input()
trDirI = trDirTIF
teDirI = teDirTIF
trWork, teWork = '../Work/Train', '../Work/Test'
trDirI,teDirI, trWork, teWork

('../Data/train-tif-v2',
 '../Data/test-tif-v2',
 '../Work/Train',
 '../Work/Test')

In [8]:
labels_df = pd.read_csv(trLabels)
labels_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [9]:
# Build list with unique labels
label_list = []
for tag_str in labels_df.tags.values:
    labels = tag_str.split(' ')
    for label in labels:
        if label not in label_list:
            label_list.append(label)

In [10]:
# Add onehot features for every label
for label in label_list:
    labels_df[label] = labels_df['tags'].apply(lambda x: 1 if label in x.split(' ') else 0)
    #labels_df[label].astype(np.int8)
# Display head
#labels_df.head()


In [11]:
weather_labels = ['clear', 'partly_cloudy', 'haze', 'cloudy']
land_labels = ['primary', 'agriculture', 'water', 'cultivation', 'habitation' ]
rare_labels = [l for l in label_list if labels_df[label_list].sum()[l] < 2000]
#rare_labels              

In [12]:
labels = label_list; #weather_labels;
nameList =labels_df[labels_df[labels].sum(axis=1)>0].image_name.tolist(); len(nameList)
labelList=labels_df[labels_df[labels].sum(axis=1)>0][labels].as_matrix();
labelList[:6,:]
#labels_df.head()

array([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [13]:
def formImExt (nf, resize=(32,32), printOK=False) :
    nx = None
    try : 
        ni = cv.imread(nf,-1); 
        if (ni is not None) :
            if not ((ni.shape[2]==3) or (ni.shape[2]==4)) and printOK : print('----- error ---- shape:',ni.shape,nf)
            if (ni.shape[2]==3) :
                nx = cv.resize(ni,resize)
            if (ni.shape[2]==4) :
                #r,g,b,n = ni[:,:,2],ni[:,:,1],ni[:,:,0],ni[:,:,3]
                r,g,b,n = cv.resize(ni[:,:,2],resize),cv.resize(ni[:,:,1],resize),cv.resize(ni[:,:,0],resize),cv.resize(ni[:,:,3],resize)
                dv,dw   = np.divide((r-n),(r+n+0.01)), np.divide((g-n),(g+n+0.01))
                nx      = np.array([r,g,b,n,dv,dw]).T; 
    except BaseException as e :
        print(nf,e); nx = None;
    
    if nx is None and printOK : 
        print('------ None:',nf); nx = None
        
    return(nx)

In [36]:
trX, trY, i, size = [],[], 0, len(nameList)
print(datetime.datetime.now(),len(nameList),size)
for nn in nameList[0:size] :
    nf = os.path.join(trDirTIF,nn+".tif");
    nx = formImExt(nf)
    if (nx is not None) :
        #rr=np.save(os.path.join(trWork,nn+".npy"),nx);
        #trX.append(nn+".npy")
        trX.append(nx)
        trY.append(True)
    else : 
        trY.append(False)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)
    #print(nn.shape)

print(datetime.datetime.now())
trX = np.array(trX);
trY = labelList[trY];
print(len(nameList),trX.shape,trY.shape)
#trXY=pd.DataFrame(trY); trXY['name']=trX; trXY.head()
#trXY.to_pickle(os.path.join(trWork,"listFiles.pkl"))

2017-05-15 01:24:39.256551 40479 40479
2017-05-15 01:25:33.407633 	 5000 	 train_4999
2017-05-15 01:27:48.406526 	 10000 	 train_9999
2017-05-15 01:30:05.273415 	 15000 	 train_14999
2017-05-15 01:32:21.207122 	 20000 	 train_19999
2017-05-15 01:34:37.987198 	 25000 	 train_24999
2017-05-15 01:36:55.296700 	 30000 	 train_29999
2017-05-15 01:39:12.678141 	 35000 	 train_34999
2017-05-15 01:41:33.065293 	 40000 	 train_39999
2017-05-15 01:41:45.582977
40479 (40479, 32, 32, 6) (40479, 17)


In [133]:
np.save('../Data-Keras/train-model-2D-1-v2-XX.npy',trX)
np.save('../Data-Keras/train-model-2D-1-v2-YY.npy',trY)

In [271]:
del(trOX); del(trOY);

In [13]:
trX = np.load('../Data-Keras/train-model-2D-1-v2-XX.npy')
trY = np.load('../Data-Keras/train-model-2D-1-v2-YY.npy')
print(trX.shape,trY.shape)

(40479, 32, 32, 6) (40479, 17)


In [14]:
import keras.backend as K
def fbeta_pred(y_true, y_pred, beta=2.0, OK1=0.2, eps=0.000001, printOK=False):
    beta2 = beta*beta
    yy_true = K.round(y_true)
    #yy_pred = K.round(y_pred+(0.5-OK1))
    yy_pred = K.round(y_pred)
    tp, tp_fp, fn = K.sum((yy_pred*yy_true)), K.sum(yy_true), K.sum((K.abs(yy_pred*(yy_true-1.0))))
    precision, recall = tp/(tp_fp+eps), tp/(tp+fn+eps) 
    fbeta = (1+beta2)*(precision*recall)/(beta2*precision+recall+eps)
    ##if fbeta>1.0 : fbeta = 1.0;
    if printOK :
        print('ten true ',K.get_value(yy_true))
        #print('ten pred ',y_pred)
        print('ten roun ',K.get_value(yy_pred))
        print(' pre=',K.get_value(precision),' recall=',K.get_value(recall),' tp=',
              K.get_value(tp),' fn=',K.get_value(fn),' tp+fp=',K.get_value(tp_fp))
    return(fbeta)

In [48]:
#
# model Krizevsky 2012
#
#    input(224x224)
#    c2d(96,(11,11),strides=(4,4))
#    c2d(256,(5,5))
#    c2d(384,(3,3))
#    c2d(384,(3,3))
#    c2d(256,(3,3))
#    maxp(2,2)
#    flaten()
#    dense(4096)
#    dense(4096)
#    dense(output(1000))
#
#
def Kriz2012 ( model, iSize, rSize ) :
    i1,i2,i3 = iSize
    model.add(BatchNormalization(input_shape=(i1,i2,i3)))
    
    model.add(Convolution2D(96,(11,11),strides=(4,4),activation='relu'))
    model.add(Convolution2D(256,(5,5),activation='relu'))
    model.add(Convolution2D(384,(3,3),activation='relu'))
    model.add(Convolution2D(384,(3,3),activation='relu'))
    model.add(Convolution2D(256,(3,3),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(rSize,activation='sigmoid'))

    return(model)

In [78]:
#
# model Krizevsky 2012
#
#    input(224x224)
#    c2d(96,(11,11),strides=(4,4))
#    c2d(256,(5,5))
#    c2d(384,(3,3))
#    c2d(384,(3,3))
#    c2d(256,(3,3))
#    maxp(2,2)
#    flaten()
#    dense(4096)
#    dense(4096)
#    dense(output(1000))
#
#
def Kriz2012 ( model, iSize, rSize ) :
    i1,i2,i3 = iSize
    model.add(BatchNormalization(input_shape=(i1,i2,i3)))
    
    model.add(Convolution2D(96,(11,11),strides=(4,4),activation='relu'))
    #model.add(Convolution2D(256,(5,5),activation='relu'))
    #model.add(Convolution2D(384,(3,3),activation='relu'))
    model.add(Convolution2D(384,(3,3),activation='relu'))
    model.add(Convolution2D(256,(3,3),activation='relu'))
    model.add(Dropout(0.25))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.5))
    ##model.add(Dense(2048,activation='relu'))
    ##model.add(Dropout(0.25))
    #model.add(Dense(4096,activation='relu'))
    #model.add(Dropout(0.25))
    
    model.add(Dense(rSize,activation='sigmoid'))

    return(model)

In [82]:
#
# try model Krizevsky 2012
#
iSize = (trX.shape[1],trX.shape[2],trX.shape[3])
#iSize = (64,64,3)
rSize = trY.shape[1]
model = Sequential()
model = Kriz2012(model,iSize,rSize)
sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', #sgd, #"adam", #'rmsprop',
              metrics=[fbeta_pred]) #['accuracy',fbeta_pred]) #['accuracy'])

In [83]:
# badly don't work: fail customs metric # model=load_model('../Data-Keras/basin02-loop-all.h5') ## verify load from v1 version
#model.load_weights('../Data-Keras/train-model-020-1D-2-v2-loop-weights.h5') ## verify load weights from v1 version

In [85]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_29 (Batc (None, 32, 32, 6)         24        
_________________________________________________________________
conv2d_110 (Conv2D)          (None, 6, 6, 96)          69792     
_________________________________________________________________
conv2d_111 (Conv2D)          (None, 4, 4, 384)         332160    
_________________________________________________________________
conv2d_112 (Conv2D)          (None, 2, 2, 256)         884992    
_________________________________________________________________
dropout_27 (Dropout)         (None, 2, 2, 256)         0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 1, 1, 256)         0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 256)               0         
__________

In [88]:
print('time.begin :',datetime.datetime.now())
#hist_old=hist.history
early_stopping = EarlyStopping(monitor='val_loss', patience=15)
hist = model.fit(trX,trY,
         epochs=100, #10,
         batch_size=256,
         validation_split=0.20,
         callbacks=[early_stopping],
         #class_weight=rrd,
         verbose=2
         )
        #samples_per_epoch=200)
        #validation_data=validation_generator,
        #initial_epoch=start,
        #nb_val_samples=2048
         #)
print('time.end   :',datetime.datetime.now())

time.begin : 2017-05-17 00:24:10.648301
Train on 32383 samples, validate on 8096 samples
Epoch 1/100
5s - loss: 4.3019 - fbeta_pred: 0.8668 - val_loss: 4.8545 - val_fbeta_pred: 0.8333
Epoch 2/100
5s - loss: 4.2883 - fbeta_pred: 0.8669 - val_loss: 4.8865 - val_fbeta_pred: 0.8363
Epoch 3/100
5s - loss: 4.2464 - fbeta_pred: 0.8705 - val_loss: 4.8953 - val_fbeta_pred: 0.8217
Epoch 4/100
5s - loss: 4.2327 - fbeta_pred: 0.8706 - val_loss: 4.9366 - val_fbeta_pred: 0.8182
Epoch 5/100
5s - loss: 4.2228 - fbeta_pred: 0.8714 - val_loss: 5.0045 - val_fbeta_pred: 0.8270
Epoch 6/100
5s - loss: 4.2005 - fbeta_pred: 0.8730 - val_loss: 4.9740 - val_fbeta_pred: 0.8240
Epoch 7/100
5s - loss: 4.1632 - fbeta_pred: 0.8775 - val_loss: 5.0350 - val_fbeta_pred: 0.8175
Epoch 8/100
5s - loss: 4.1463 - fbeta_pred: 0.8803 - val_loss: 5.0059 - val_fbeta_pred: 0.8174
Epoch 9/100
5s - loss: 4.1450 - fbeta_pred: 0.8783 - val_loss: 5.0728 - val_fbeta_pred: 0.8244
Epoch 10/100
5s - loss: 4.1190 - fbeta_pred: 0.8831 - va

In [251]:
save_model(model,'../Data-Keras/train-model-2D-2-v2-all.h5')
model.save_weights('../Data-Keras/train-model-2D-2-v2-weights.h5')  # save weights after training

In [54]:
print(model.evaluate(trX,trY,verbose=2))
trP = model.predict(trX, batch_size=128)
print(trY.shape)
print(trP.shape)
print('fbeta_score=',fbeta_score(trY, np.array(trP) > 0.2, beta=2, average='samples'))
print('fbeta_pred =',K.get_value(fbeta_pred(trY.astype(np.float64),trP.astype(np.float64))))

[4.3339444657595294, 0.71995355617081491, 0.87297436628756564]
(40479, 17)
(40479, 17)
fbeta_score= 0.900652762317
fbeta_pred = 0.872814040159


In [89]:
print(model.evaluate(trX,trY,verbose=2))
trP = model.predict(trX, batch_size=512)
print(trY.shape)
print(trP.shape)
print('fbeta_score=',fbeta_score(trY, np.array(trP) > 0.2, beta=2, average='samples'))
print('fbeta_pred =',K.get_value(fbeta_pred(trY.astype(np.float64),trP.astype(np.float64))))

[4.1305325137004809, 0.89616915144884179]
(40479, 17)
(40479, 17)
fbeta_score= 0.922396254017
fbeta_pred = 0.896171333129


In [286]:
#hist_old=hist.history

In [48]:
print('time.begin :',datetime.datetime.now())
beta_old = 0.0
trP = model.predict(trX, batch_size=512)
beta_old = fbeta_score(trY, np.array(trP) > 0.2, beta=2, average='samples')
print(datetime.datetime.now(),0,beta_old,model.evaluate(trX,trY,verbose=2))
for i in range(10) :
    model.fit(trX,trY,
             nb_epoch=10,
             batch_size=128,
             validation_split=0.20,
             verbose=0
             )
    trP = model.predict(trX, batch_size=512)
    beta_new = fbeta_score(trY, np.array(trP) > 0.2, beta=2, average='samples')
    print(datetime.datetime.now(),i,beta_new,"(",beta_old,")",model.evaluate(trX,trY,verbose=2))
    if ((beta_old+0.003)<beta_new) :
        save_model(model,'../Data-Keras/train-model-2D-2-v2-loop-all.h5')
        model.save_weights('../Data-Keras/train-model-2D-2-v2-loop-weights.h5')  # save weights after training
        beta_old = beta_new
print('time.end   :',datetime.datetime.now())

time.begin : 2017-05-15 01:58:40.818427
2017-05-15 01:58:43.986029 0 0.868059261389 [4.6235675547191812, 0.85972495279769423]
2017-05-15 01:59:55.872659 0 0.867757843378 ( 0.868059261389 ) [4.634846947510554, 0.85930002745810685]
2017-05-15 02:01:07.827121 1 0.865095514463 ( 0.868059261389 ) [4.6301586407264432, 0.86307829108238576]
2017-05-15 02:02:20.435411 2 0.872059892074 ( 0.868059261389 ) [4.5869103705371481, 0.86574812870481044]
2017-05-15 02:03:32.269407 3 0.87089557615 ( 0.872059892074 ) [4.592559225000457, 0.86118734666265084]
2017-05-15 02:04:44.437900 4 0.865811295641 ( 0.872059892074 ) [4.6107854238361377, 0.86119799125206031]
2017-05-15 02:05:56.442050 5 0.874604359252 ( 0.872059892074 ) [4.5657333543629939, 0.86629773470431837]
2017-05-15 02:07:08.406273 6 0.874211744714 ( 0.872059892074 ) [4.5562372201987262, 0.86594739223163253]
2017-05-15 02:08:19.845920 7 0.86667917166 ( 0.872059892074 ) [4.580335893008769, 0.86347966977631663]
2017-05-15 02:09:32.147029 8 0.86713749

In [252]:
rr, rrx = [], []; trP = model.predict(trX, batch_size=512)
for i in range(trP.shape[1]) :
    xx = [];
    trYY = trY[:,i].astype(np.float64)
    trPY = trP[:,i]
    for ii in range(100) :
        trPP = (trPY>0.01*ii).astype(np.float64)
        #x = fbeta_score(trY[:,0], np.array(trP[:,0] > 0.1*ii), beta=2, average='samples')
        x = K.get_value(fbeta_pred(trYY,trPP))
        xx.append(x)
    rrr = np.array(xx).argmax();
    rr.append(rrr)
    rrx.append(xx[rrr])
    print(datetime.datetime.now(),i,'-->',rrr,xx[rrr],xx[(rrr-1):(rrr+2)])
    #print(xx);
    #plt.plot(np.array(xx)); plt.show()
trM = np.array(rr)/100.0;

2017-05-15 07:45:16.245791 0 --> 67 0.703723037192 [0.70136893207458639, 0.70372303719187324, 0.70255697093345204]
2017-05-15 07:48:05.845385 1 --> 74 0.980069323901 [0.97993962183479821, 0.98006932390084267, 0.97997932676465371]
2017-05-15 07:51:03.555375 2 --> 60 0.830832805721 [0.83066919227432678, 0.83083280572116081, 0.83012642884749333]
2017-05-15 07:54:07.692849 3 --> 73 0.95596621068 [0.95530247952573877, 0.9559662106804423, 0.95587193006934856]
2017-05-15 07:57:17.654396 4 --> 37 0.771627389839 [0.77128836874582085, 0.77162738983933221, 0.77157084802598352]
2017-05-15 08:00:35.571387 5 --> 28 0.612647945213 [0.6089740900094327, 0.61264794521337496, 0.60631829415658134]
2017-05-15 08:04:02.404595 6 --> 40 0.711618933993 [0.70809511238058165, 0.71161893399277543, 0.71149522396880938]
2017-05-15 08:07:36.162990 7 --> 38 0.549023191429 [0.54805400909065172, 0.54902319142904643, 0.54874345592600848]
2017-05-15 08:11:16.278601 8 --> 20 0.24951985087 [0.24604540154654581, 0.249519850

In [253]:
print(trM,'\n',np.array(labels),'\n',rrx,'\n',(np.array(rrx)).mean())

[ 0.67  0.74  0.6   0.73  0.37  0.28  0.4   0.38  0.2   0.75  0.39  0.23
  0.19  0.36  0.08  0.3   0.05] 
 ['haze' 'primary' 'agriculture' 'clear' 'water' 'habitation' 'road'
 'cultivation' 'slash_burn' 'cloudy' 'partly_cloudy' 'conventional_mine'
 'bare_ground' 'artisinal_mine' 'blooming' 'selective_logging' 'blow_down'] 
 [0.70372303719187324, 0.98006932390084267, 0.83083280572116081, 0.9559662106804423, 0.77162738983933221, 0.61264794521337496, 0.71161893399277543, 0.54902319142904643, 0.24951985087034834, 0.80765954376652815, 0.89400227748987626, 0.35326054982895766, 0.30523953300436946, 0.79442485880374014, 0.15415527920611241, 0.26632276444121183, 0.082644325183804948] 
 0.589572812974


In [69]:
print(trM,'\n',np.array(labels),'\n',rrx,'\n',(np.array(rrx)).mean())

[ 0.64  0.84  0.67  0.92  0.47  0.42  0.57  0.45  0.21  0.98  0.6   0.15
  0.33  0.46  0.05  0.18  0.05] 
 ['haze' 'primary' 'agriculture' 'clear' 'water' 'habitation' 'road'
 'cultivation' 'slash_burn' 'cloudy' 'partly_cloudy' 'conventional_mine'
 'bare_ground' 'artisinal_mine' 'blooming' 'selective_logging' 'blow_down'] 
 [0.69066801422279211, 0.98119478181654729, 0.81726124420066348, 0.95900718919257255, 0.78940066739193948, 0.5789222121703973, 0.70371154814742987, 0.56662533268054449, 0.25948073141178507, 0.78421972946080287, 0.91042787787543189, 0.25862037752711825, 0.29865100169393016, 0.75683228670815861, 0.10067083166393269, 0.23247956280989218, 0.176990839533778] 
 0.580303778148


In [90]:
save_model(model,  '../Data-Keras/train-model-2D-2-v2-loop-all.h5')
model.save_weights('../Data-Keras/train-model-2D-2-v2-loop-weights.h5')  # save weights after training

In [91]:
# Forming output dataset for predicting --> trOX, trOY
del(trX)
del(trY)

In [116]:
nameAsk = os.listdir(teDirI); print(len(nameAsk))
trOX, trOY, i, size = [], [], 0, len(nameAsk)
print(datetime.datetime.now(),len(nameList),size)
for nn in nameAsk[0:size] :
    nf = os.path.join(teDirTIF,nn);
    nx = formImExt(nf)
    if (nx is not None) :
        trOX.append(nx)
        trOY.append(nn)
    i += 1
    if (i%5000==0) : print(datetime.datetime.now(),"\t",i,"\t",nn)

print(datetime.datetime.now())

#assert (size!=len(trOY)), "Wrong files {} != {}".format(size,len(trOY))

61191
2017-05-15 03:41:31.480157 40479 61191
2017-05-15 03:44:42.278867 	 5000 	 test_15649.tif
2017-05-15 03:47:38.148162 	 10000 	 test_32423.tif
2017-05-15 03:50:30.404013 	 15000 	 test_36264.tif
2017-05-15 03:53:15.246494 	 20000 	 test_10894.tif
2017-05-15 03:56:02.300335 	 25000 	 file_9502.tif
2017-05-15 03:58:41.448958 	 30000 	 file_14128.tif
2017-05-15 04:01:16.200243 	 35000 	 file_17389.tif
2017-05-15 04:03:46.526075 	 40000 	 test_19356.tif
2017-05-15 04:06:14.270358 	 45000 	 test_10134.tif
2017-05-15 04:08:39.599717 	 50000 	 test_24243.tif
2017-05-15 04:11:03.067190 	 55000 	 test_7049.tif
2017-05-15 04:13:24.769720 	 60000 	 file_10482.tif
2017-05-15 04:13:56.596995


In [117]:
trOX = np.array(trOX);
trOY = np.array([os.path.splitext(x)[0] for x in trOY]);
print(len(nameAsk),trOX.shape,trOY.shape)

61191 (61191, 32, 32, 6) (61191,)


In [ ]:
# Saving & Loading

In [118]:
np.save('../Data-Keras/test-basin-2D-1-OX-tif-v2.npy',trOX)
np.save('../Data-Keras/test-basin-2D-1-OY-tif-v2.npy',trOY)

In [92]:
trOX = np.load('../Data-Keras/test-basin-2D-1-OX-tif-v2.npy')
trOY = np.load('../Data-Keras/test-basin-2D-1-OY-tif-v2.npy')
print(trOX.shape,trOY.shape)

(61191, 32, 32, 6) (61191,)


In [93]:
rrx
rrd=dict()
for i in range(len(rrx)) :
    rrd[i]=rrx[i]
rrd

NameError: name 'rrx' is not defined

In [258]:
#Forming result

In [94]:
trP = model.predict(trOX, batch_size=512); 
trP.shape, trP[-1]

((61191, 17), array([  2.31444892e-02,   9.79294121e-01,   2.86209971e-01,
          9.69578385e-01,   3.21317136e-01,   3.53857726e-02,
          1.92450643e-01,   1.09134607e-01,   5.12446323e-03,
          7.43655721e-04,   2.26537101e-02,   7.24586775e-04,
          1.53525993e-02,   2.39399401e-03,   2.09098756e-02,
          2.87257116e-02,   1.37278186e-02], dtype=float32))

In [96]:
trM=np.array([0.0]*len(labels)); print(trM.shape)
wr = [labels.index(i) for i in weather_labels];
tt=trM; trM[:]=0.5
trM[np.array(wr)] = 2.0
print(wr,'\n',labels,'\n',trM.tolist())

(17,)
[3, 10, 0, 9] 
 ['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down'] 
 [2.0, 0.5, 0.5, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 2.0, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


In [97]:
#trP = model.predict(trX, batch_size=512); trP=K.get_value(trP)
res = []

for i in range(trP.shape[0]) :
    trPP = [weather_labels[trP[i,wr].argmax()]] + [labels[ii] for ii in range(len(labels)) if (trP[i,ii]>trM[ii])];
    pp   = ' '.join(trPP)
    ##if (pp=="") : print(trY[i])
    res.append([trOY[i],pp])

res.sort(cmp=lambda x,y: cmp(int(x[0].partition('_')[2]),int(y[0].partition('_')[2])) if (x[0].partition('_')[0]==y[0].partition('_')[0]) else cmp(y[0].partition('_')[0],x[0].partition('_')[0]))
#print(res[4:8],'\n',res[-4:])
print(res[0],'\n',res[-1],'\n',trM,trP[0])

['test_0', 'clear primary'] 
 ['file_20521', 'clear primary agriculture cultivation'] 
 [ 2.   0.5  0.5  2.   0.5  0.5  0.5  0.5  0.5  2.   2.   0.5  0.5  0.5  0.5
  0.5  0.5] [  1.87955244e-04   9.99488473e-01   1.02253677e-02   9.99854803e-01
   4.71423054e-03   6.42386824e-03   7.78392749e-03   3.67686129e-03
   5.46602814e-06   1.14584999e-07   1.21952980e-04   1.72610100e-07
   3.21809166e-05   4.81529874e-08   6.05761074e-03   2.31929217e-03
   8.98394641e-03]


In [266]:
print(labels)
print(trM.tolist())
np.round(trP[4:11,:])

['haze', 'primary', 'agriculture', 'clear', 'water', 'habitation', 'road', 'cultivation', 'slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine', 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down']
[2.0, 0.5, 0.5, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 2.0, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]


array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.],
       [ 0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.]], dtype=float32)

In [98]:
rrr=pd.DataFrame(res,columns=['image_name','tags']); rrr.head(); 
suffixDT = (datetime.datetime.now()).strftime('%Y-%m-%d-%H-%M-%S'); print(suffixDT)
rrr.to_csv('../Result/vss'+suffixDT+'.csv',index=False);

2017-05-17-00-33-31
